In [ ]:
%pip install -q -U transformers flash-attn

In [ ]:
import torch
from modelling_llava import LlavaForCausalLM

model = LlavaForCausalLM.from_pretrained(
    "visheratin/MC-LLaVA-3b", torch_dtype=torch.float16, trust_remote_code=True
).to("cuda")

In [ ]:
from transformers import AutoProcessor
from peft import PeftModel, LoraConfig

processor = AutoProcessor.from_pretrained(
    "visheratin/MC-LLaVA-3b", trust_remote_code=True
)

lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=["k_proj", "q_proj", "v_proj", "out_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=None,
)
model = PeftModel(model, lora_config)

In [ ]:
state_dict = torch.load(
    "checkpoints_ft_mc_llava/checkpoint_3000.pt", map_location="cuda"
)
model.load_state_dict(state_dict, strict=True)


In [ ]:
class_artifact = {
    "airplane": [
        "Artificial noise patterns in uniform surfaces",
        "Metallic surface artifacts",
        "Impossible mechanical connections",
        "Inconsistent scale of mechanical parts",
        "Physically impossible structural elements",
        "Implausible aerodynamic structures",
        "Misaligned body panels",
        "Impossible mechanical joints",
        "Distorted window reflections",
    ],
    "automobile": [
        "Artificial noise patterns in uniform surfaces",
        "Metallic surface artifacts",
        "Impossible mechanical connections",
        "Inconsistent scale of mechanical parts",
        "Physically impossible structural elements",
        "Incorrect wheel geometry",
        "Misaligned body panels",
        "Impossible mechanical joints",
        "Distorted window reflections",
    ],
    "ship": [
        "Artificial noise patterns in uniform surfaces",
        "Metallic surface artifacts",
        "Impossible mechanical connections",
        "Inconsistent scale of mechanical parts",
        "Physically impossible structural elements",
        "Misaligned body panels",
    ],
    "truck": [
        "Artificial noise patterns in uniform surfaces",
        "Metallic surface artifacts",
        "Impossible mechanical connections",
        "Inconsistent scale of mechanical parts",
        "Physically impossible structural elements",
        "Incorrect wheel geometry",
        "Misaligned body panels",
        "Impossible mechanical joints",
        "Distorted window reflections",
    ],
    "bird": [
        "Unrealistic eye reflections",
        "Misshapen ears or appendages",
        "Anatomically impossible joint configurations",
        "Unnatural pose artifacts",
        "Biological asymmetry errors",
        "Regular grid-like artifacts in textures",
        "Impossible foreshortening in animal bodies",
        "Misaligned bilateral elements in animal faces",
        "Over-smoothing of natural textures",
    ],
    "cat": [
        "Unrealistic eye reflections",
        "Misshapen ears or appendages",
        "Anatomically impossible joint configurations",
        "Unnatural pose artifacts",
        "Biological asymmetry errors",
        "Regular grid-like artifacts in textures",
        "Impossible foreshortening in animal bodies",
        "Misaligned bilateral elements in animal faces",
        "Over-smoothing of natural textures",
        "Anatomically incorrect paw structures",
        "Improper fur direction flows",
    ],
    "deer": [
        "Unrealistic eye reflections",
        "Misshapen ears or appendages",
        "Anatomically impossible joint configurations",
        "Unnatural pose artifacts",
        "Biological asymmetry errors",
        "Regular grid-like artifacts in textures",
        "Impossible foreshortening in animal bodies",
        "Misaligned bilateral elements in animal faces",
        "Over-smoothing of natural textures",
        "Improper fur direction flows",
    ],
    "dog": [
        "Unrealistic eye reflections",
        "Misshapen ears or appendages",
        "Anatomically impossible joint configurations",
        "Unnatural pose artifacts",
        "Biological asymmetry errors",
        "Regular grid-like artifacts in textures",
        "Impossible foreshortening in animal bodies",
        "Misaligned bilateral elements in animal faces",
        "Over-smoothing of natural textures",
        "Dental anomalies in mammals",
        "Anatomically incorrect paw structures",
        "Improper fur direction flows",
    ],
    "frog": [
        "Unrealistic eye reflections",
        "Misshapen ears or appendages",
        "Anatomically impossible joint configurations",
        "Unnatural pose artifacts",
        "Biological asymmetry errors",
        "Regular grid-like artifacts in textures",
        "Impossible foreshortening in animal bodies",
        "Misaligned bilateral elements in animal faces",
        "Over-smoothing of natural textures",
    ],
    "horse": [
        "Unrealistic eye reflections",
        "Misshapen ears or appendages",
        "Anatomically impossible joint configurations",
        "Unnatural pose artifacts",
        "Biological asymmetry errors",
        "Regular grid-like artifacts in textures",
        "Impossible foreshortening in animal bodies",
        "Misaligned bilateral elements in animal faces",
        "Over-smoothing of natural textures",
        "Dental anomalies in mammals",
    ],
    "major": [
        "Discontinuous surfaces",
        "Non-manifold geometries in rigid structures",
        "Asymmetric features in naturally symmetric objects",
        "Texture bleeding between adjacent regions",
        "Excessive sharpness in certain image regions",
        "Artificial smoothness",
        "Movie-poster-like composition of ordinary scenes",
        "Unnatural lighting gradients",
        "Fake depth of field",
        "Abruptly cut-off objects",
        "Color coherence breaks",
        "Spatial relationship errors",
        "Depth perception anomalies",
        "Over-sharpening artifacts",
        "Incorrect reflection mapping" "Inconsistent object boundaries",
        "Floating or disconnected components",
        "Texture repetition patterns",
        "Unrealistic specular highlights",
        "Inconsistent material properties",
        "Inconsistent shadow directions",
        "Multiple light source conflicts",
        "Missing ambient occlusion",
        "Incorrect perspective rendering",
        "Scale inconsistencies within single objects",
        "Aliasing along high-contrast edges",
        "Blurred boundaries in fine details",
        "Jagged edges in curved structures",
        "Random noise patterns in detailed areas",
        "Loss of fine detail in complex structures",
        "Artificial enhancement artifacts",
        "Repeated element patterns",
        "Systematic color distribution anomalies",
        "Frequency domain signatures",
        "Unnatural color transitions",
        "Resolution inconsistencies within regions",
        "Glow or light bleed around object boundaries",
        "Ghosting effects: Semi-transparent duplicates of elements",
        "Cinematization effects",
        "Dramatic lighting that defies natural physics",
        "Artificial depth of field in object presentation",
        "Unnaturally glossy surfaces",
        "Synthetic material appearance",
        "Multiple inconsistent shadow sources",
        "Exaggerated characteristic features",
        "Scale inconsistencies within the same object class" "Incorrect skin tones",
    ],
}

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt


def generate(image_url, max_crops=500, num_tokens=768, label="horse"):
    raw_image = Image.open(image_url).convert("RGB")
    # plt.imshow(raw_image)
    # plt.show()

    question = f"""Analyze the provided image to identify and explain distinguishing artifacts that indicate it is fake. Provide clear, concise explanations (maximum 50 words each) using the specified artifacts below. Include positional references like 'top left' or 'bottom right' when relevant. DO NOT include any other sentences or artifacts in your response.\nExample output: \nResolution and Detail: Missing fine details like individual hairs, replaced by smooth texture\nAnatomical Inconsistencies: Distorted leg proportions and joint angles\nONLY use the artifacts listed below: \n {class_artifact[label]+class_artifact['major']}"""

    prompt = prompt = f"""<|im_start|>user
                <image>
                {question}<|im_end|>
                <|im_start|>assistant
                """

    with torch.inference_mode():
        inputs = processor(
            prompt, [raw_image], model, max_crops=max_crops, num_tokens=num_tokens
        )
        inputs["input_ids"] = inputs["input_ids"].to(model.device)
        inputs["attention_mask"] = inputs["attention_mask"].to(model.device)

        output = model.generate(
            **inputs,
            max_new_tokens=250,
            use_cache=True,
            do_sample=True,
            temperature=0.8,
        )
        output_text = processor.batch_decode(output, skip_special_tokens=True)

        return output_text

In [ ]:
import pandas as pd

df = pd.read_csv("data.csv")

for i, row in df.iterrows():
    image_url = row["image_url"]

    output_text = generate(image_url, label=row["label"])


In [ ]:
df.to_csv("mc_llava_output.csv", index=False)